In [1]:
using constants
using types
using Optim
using FastAnonymous
E_lowlimit=-1.0
E_highlimit=1.0
interval=0.01
xmax=3.0
Temp=300.0
Ef=0.8
E=0.7
Erange=collect(E_lowlimit:interval:E_highlimit)
band1=parBand(1.0*me,0.0,0.5)
band2=parBand(-2.0*me,0.0,-0.2)
bndst=BandStruc([band1,band2])
function getDOS_Epoint(band::parBand,E)
     if band.effMass<0.0 
            if -E-band.offset>=0            
            dos=real(im*(pi*8*sqrt(2).*Complex(band.effMass).^(3.0/2.0).*sqrt((-E-band.offset).*q)/h/h/h))
             else
                 dos=0.0
             end
        end
        if band.effMass>0.0 
            if E>=band.offset         
               dos=real((pi*8*sqrt(2).*Complex(band.effMass).^(3.0/2.0).*sqrt((E-band.offset)*q)/h/h/h))
             else
                dos=0.0
             end
        end
    return dos
end
function fermiStatpoint(Temp,Ef,E::Float64)
   return 1.0./(1.0+exp(q*(E-Ef)./kB./Temp)) 
end
function Numofnout(band::parBand,Ef,Temp,xmax::Float64)
    if band.effMass>0.0
        integrand(E)=q*getDOS_Epoint(band,E).*(-1*fermiStatpoint(Temp,Ef,E))
        a=quadgk(integrand,-1*xmax,xmax)
    else
        integrand(E)=q*getDOS_Epoint(band,E).*(1-fermiStatpoint(Temp,Ef,E))
        a=quadgk(integrand,-1*xmax,xmax)
    end
    
    return a[1]
end
function FindFermiLevel(numberofn::Float64,band::parBand,Ef,Temp::Float64,xmax::Float64)
    return abs(numberofn*1e6-Numofnout(band,Ef,Temp,xmax))
end
function NumofnMultiBand(bndst::BandStruc,Ef,Temp,xmax::Float64)
    totalnumofn=0.0
    for band in bndst.bands
        totalnumofn=totalnumofn+Numofnout(band,Ef,Temp,xmax) 
    end
    return totalnumofn 
end
function FindFermiLevel(numberofn::Float64,bndst::BandStruc,Ef,Temp,xmax::Float64)
    return abs(numberofn*1e6-NumofnMultiBand(bndst,Ef,Temp,xmax))
end
println("getdos_EPoint called")
@time dos=getDOS_Epoint(band1,E)
println("DOS at $E is $dos")
println("fermiStatpoint called")
@time fSp=fermiStatpoint(Temp,Ef,E)
println("Fermi at $Temp, $Ef, $E is $fSp")
println("Numofn is called")
@time numofnoutput=Numofnout(band1,Ef,Temp,xmax)/1e6
println("Num. of n at $Temp, $Ef, $E is $numofnoutput")
numofninput=numofnoutput
equation(x)=FindFermiLevel(numofninput,band1,x,Temp,xmax)
println("optimize is called")
@time Efcalc=optimize(equation,-1.0,1.0)#method=:golden_section)
println("optimize result is ", Efcalc.minimum)
equation(x)=FindFermiLevel(numofninput,bndst,x,Temp,xmax)
println("optimize for multiband is called")
@time EfcalcM=optimize(equation,-1.0,1.0,method=:golden_section)
println("optimize result for multiband is", EfcalcM.minimum)

getdos_EPoint called
  0.095037 seconds (141.48 k allocations: 5.929 MB)
DOS at 0.7 is 1.9015342619863263e46
fermiStatpoint called
  0.006387 seconds (6.00 k allocations: 314.438 KB)
Fermi at 300.0, 0.8, 0.7 is 0.9795312041510806
Numofn is called
  0.465801 seconds (772.59 k allocations: 37.091 MB, 2.24% gc time)
Num. of n at 300.0, 0.8, 0.7 is -7.5314409993024e20
optimize is called
  0.337695 seconds (1.01 M allocations: 32.409 MB, 1.60% gc time)
optimize result is 0.800000002561175
optimize for multiband is called
431.161368 seconds (8.68 G allocations: 160.037 GB, 25.19% gc time)
optimize result for multiband is0.7999999894328761


In [52]:
using constants
using types
using Optim
using FastAnonymous
E_lowlimit=-1.0
E_highlimit=1.0
interval=0.01
xmax=3.0
Temp=300.0
Ef=0.95
E=0.7
Erange=collect(E_lowlimit:interval:E_highlimit)
band1=parBand(1.0*me,0.0,0.5)
band2=parBand(-2.0*me,0.0,-0.2)
bndst=BandStruc([band1,band2])
function getDOS_Epoint(band::parBand,E)
     if band.effMass<0.0 
            if -E-band.offset>=0            
            dos=real(im*(pi*8*sqrt(2).*Complex(band.effMass).^(3.0/2.0).*sqrt((-E-band.offset).*q)/h/h/h))
             else
                 dos=0.0
             end
        end
        if band.effMass>0.0 
            if E>=band.offset         
               dos=real((pi*8*sqrt(2).*Complex(band.effMass).^(3.0/2.0).*sqrt((E-band.offset)*q)/h/h/h))
             else
                dos=0.0
             end
        end
    return dos
end
function fermiStatpoint(Temp,Ef,E::Float64)
   return 1.0./(1.0+exp(q*(E-Ef)./kB./Temp)) 
end
function Numofnout(band::parBand,Ef,Temp,xmax::Float64)
    if band.effMass>0.0
        integrand(E)=q*getDOS_Epoint(band,E).*(-1*fermiStatpoint(Temp,Ef,E))
        a=quadgk(integrand,Ef-2kBe*Temp,Ef+2kBe*Temp)
        #println("Numofn >0 Band eff mass is ",band.effMass)
    else
        #println("Numofn <0 Band eff mass is ",band.effMass)
        integrand(E)=q*getDOS_Epoint(band,E).*(1-fermiStatpoint(Temp,Ef,E))
        a=quadgk(integrand,Ef-2kBe*Temp,Ef+2kBe*Temp)
    end
    
    return a[1]
end
function FindFermiLevel(numberofn::Float64,band::parBand,Ef,Temp::Float64,xmax::Float64)
    return abs(numberofn*1e6-Numofnout(band,Ef,Temp,xmax))
end
function NumofnMultiBand(bndst::BandStruc,Ef,Temp,xmax::Float64)
    totalnumofn=0.0
    for band in bndst.bands
        totalnumofn=totalnumofn+Numofnout(band,Ef,Temp,xmax) 
    end
    return totalnumofn 
end
function FindFermiLevel(numberofn::Float64,bndst::BandStruc,Ef,Temp,xmax::Float64)
    return abs(numberofn*1e6-NumofnMultiBand(bndst,Ef,Temp,xmax))
end
#println("getdos_EPoint called")
#@time dos=getDOS_Epoint(band1,E)
#println("DOS at $E is $dos")
#println("fermiStatpoint called")
#@time fSp=fermiStatpoint(Temp,Ef,E)
#println("Fermi at $Temp, $Ef, $E is $fSp")
println("Numofn for band1 is called")
@time numofnoutput1=Numofnout(band1,Ef,Temp,xmax)/1e6
println("Num. of n for band1 at $Temp, $Ef, $E is $numofnoutput1")
println("Numofn for band2 is called")
@time numofnoutput2=Numofnout(band2,Ef,Temp,xmax)/1e6
println("Num. of n for band2 at $Temp, $Ef, $E is $numofnoutput2")
numofninput1=numofnoutput1
numofninput2=numofnoutput2
numofninput=numofninput1+numofninput2
equation(x)=FindFermiLevel(numofninput1,band1,x,Temp,xmax)
#println("optimize is called")
#@time Efcalc=optimize(equation,-1.0,1.0)#method=:golden_section)
#println("Original Ef was $Ef optimize result is ", Efcalc.minimum)
println("NumofnMulti is called")
@time numofnoutputMulti=NumofnMultiBand(bndst,Ef,Temp,xmax)/1e6
println("NumMulti. of n at $Temp, $Ef, $E is $numofnoutputMulti")
numofninputMulti=numofnoutputMulti
equation(x)=FindFermiLevel(numofninputMulti,bndst,x,Temp,xmax)
println("optimize for multiband is called")
@time EfcalcM=optimize(equation,-1.0,1.0,method=:golden_section)
println("optimize result for multiband is ", EfcalcM.minimum)
message= EfcalcM.converged ? "sucess" : "problem"
println(message)

Numofn for band1 is called
  0.050408 seconds (8.72 k allocations: 415.887 KB)
Num. of n for band1 at 300.0, 0.95, 0.7 is -2.3233048688347152e20
Numofn for band2 is called
  0.002478 seconds (1.13 k allocations: 51.921 KB)
Num. of n for band2 at 300.0, 0.95, 0.7 is 0.0
NumofnMulti is called
  0.006236 seconds (4.22 k allocations: 190.854 KB)
NumMulti. of n at 300.0, 0.95, 0.7 is -2.3233048688347152e20
optimize for multiband is called
  0.011314 seconds (47.75 k allocations: 953.504 KB)
optimize result for multiband is 0.949999995173154
sucess


In [4]:
f(x)=x^2
g(x)=x^3
H=[f,g]
function funcArray(H::Array{Function})
    total=0
    for func in H
        total+=func(2)
    end
    return total
end
x=funcArray(H)

12

In [6]:
type taaa
    f::Function
     
end
function test(x)
    return x*x 
end
t=taaa(test)
t.f(2)

4

In [22]:
using constants
a=kBe*300.0

0.025851997199999997